In [96]:
import numpy as np
import cv2
from matplotlib import pyplot as plt

In [97]:
IMAGE_REFERENCE = 'earth_img\photo_092_53245529093_o.jpg'
IMAGE_COMPARISON = 'earth_img\photo_091_53245728575_o.jpg'

In [98]:
def remove_outliers_ransac(keypoints1, keypoints2, matches):
    """
    Remove outliers using RANSAC to compute homography.
    """
    src_pts = np.float32([keypoints1[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
    dst_pts = np.float32([keypoints2[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)
    
    # Calculate the Homography matrix with RANSAC
    M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
    
    return M, mask

In [99]:
def match_sift(image):
    sift_detector = cv2.SIFT_create()
    kp1, des1 = sift_detector.detectAndCompute(image, None)
    return kp1, des1

def match_bf(des1, des2):
    bf = cv2.BFMatcher()
    matches = bf.knnMatch(des1, des2, k=2)
    return matches

def match_flann(des1, des2):
    FLANN_INDEX_KDTREE = 0
    index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
    search_params = dict(checks=50)
    flann = cv2.FlannBasedMatcher(index_params, search_params)
    matches = flann.knnMatch(des1, des2, k=2)
    return matches

def match_ratio(matches, ratio):
    good = []
    for m, n in matches:
        if m.distance < ratio * n.distance:
            good.append(m)
    return good

def draw_matches(image1, kp1, image2, kp2, matches):
    h1, w1 = image1.shape[:2]
    h2, w2 = image2.shape[:2]
    vis = np.zeros((max(h1, h2), w1 + w2, 3), np.uint8)
    vis[:h1, :w1] = image1
    vis[:h2, w1:w1 + w2] = image2
    for m in matches:
        pt1 = (int(kp1[m.queryIdx].pt[0]), int(kp1[m.queryIdx].pt[1]))
        pt2 = (int(kp2[m.trainIdx].pt[0]) + w1, int(kp2[m.trainIdx].pt[1]))
        cv2.line(vis, pt1, pt2, (0, 255, 0), 1)
    return vis

def main():
    image1 = cv2.imread(IMAGE_COMPARISON, cv2.IMREAD_GRAYSCALE)
    image2 = cv2.imread(IMAGE_REFERENCE, cv2.IMREAD_GRAYSCALE)
    # Convert grayscale images to color
    image1_color = cv2.cvtColor(image1, cv2.COLOR_GRAY2BGR)
    image2_color = cv2.cvtColor(image2, cv2.COLOR_GRAY2BGR)
    kp1, des1 = match_sift(image1)
    kp2, des2 = match_sift(image2)
    matches = match_bf(des1, des2)
    print(len(matches))
    #matches = match_ratio(matches, 0.75)
    print(remove_outliers_ransac(kp1, kp2, matches))
    vis = draw_matches(image1_color, kp1, image2_color, kp2, matches)
    plt.imshow(cv2.cvtColor(vis, cv2.COLOR_BGR2RGB))
    plt.title("Matches")
    plt.show()

main()


1020


AttributeError: 'tuple' object has no attribute 'queryIdx'

In [110]:
def remove_outliers_ransac(keypoints1, keypoints2, matches):
    """
    Remove outliers using RANSAC to compute homography.
    """
    # Extract keypoints as float32
    src_pts = np.float32([keypoints1[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
    dst_pts = np.float32([keypoints2[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)
    
    # Calculate the Homography matrix with RANSAC
    M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
    matches_mask = mask.ravel().tolist()
    
    # Filter matches based on RANSAC mask (1 = inlier, 0 = outlier)
    inliers = [m for i, m in enumerate(matches) if matches_mask[i]]

    mean_displacement = []
    
    # Calculate the mean displacement vector for inliers
    for m in inliers:
        pt1 = keypoints1[m.queryIdx].pt
        pt2 = keypoints2[m.trainIdx].pt
        dx = pt2[0] - pt1[0]
        dy = pt2[1] - pt1[1]
        mean_displacement.append((dx, dy))

    # Compute the mean displacement vector
    mean_displacement = np.mean(mean_displacement, axis=0)

    print(f"Mean Displacement Vector: dx={mean_displacement[0]}, dy={mean_displacement[1]}")

    return M, inliers

def match_sift(image):
    sift_detector = cv2.SIFT_create()
    kp1, des1 = sift_detector.detectAndCompute(image, None)
    return kp1, des1

def match_flann(des1, des2):
    FLANN_INDEX_KDTREE = 1
    index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
    search_params = dict(checks=50)
    flann = cv2.FlannBasedMatcher(index_params, search_params)
    matches = flann.knnMatch(des1, des2, k=2)
    return matches

def lowe_ratio_test(matches, ratio=0.75):
    good_matches = []
    for m, n in matches:
        if m.distance < ratio * n.distance:
            good_matches.append(m)
    return good_matches

def match_bf(des1, des2):
    bf = cv2.BFMatcher()
    matches = bf.knnMatch(des1, des2, k=2)
    
    # Keep only the best matches (the ones with the smallest distance)
    good_matches = []
    for m, n in matches:
        if m.distance < 0.75 * n.distance:
            good_matches.append(m)
    
    return good_matches

def draw_matches(image1, kp1, image2, kp2, matches):
    h1, w1 = image1.shape[:2]
    h2, w2 = image2.shape[:2]
    vis = np.zeros((max(h1, h2), w1 + w2, 3), np.uint8)
    vis[:h1, :w1] = image1
    vis[:h2, w1:w1 + w2] = image2
    for m in matches:
        pt1 = (int(kp1[m.queryIdx].pt[0]), int(kp1[m.queryIdx].pt[1]))
        pt2 = (int(kp2[m.trainIdx].pt[0]) + w1, int(kp2[m.trainIdx].pt[1]))
        cv2.line(vis, pt1, pt2, (0, 255, 0), 1)
    return vis

def main():
    image1 = cv2.imread(IMAGE_COMPARISON, cv2.IMREAD_GRAYSCALE)
    image2 = cv2.imread(IMAGE_REFERENCE, cv2.IMREAD_GRAYSCALE)
    
    # Preprocess the images and rescale
    image1, img1_shape = preprocess_image(IMAGE_COMPARISON, scale_factor=0.5)
    image2, img2_shape = preprocess_image(IMAGE_REFERENCE, scale_factor=0.5)

    # Convert grayscale images to color for visualization
    image1_color = cv2.cvtColor(image1, cv2.COLOR_GRAY2BGR)
    image2_color = cv2.cvtColor(image2, cv2.COLOR_GRAY2BGR)
    
    # Detect keypoints using SIFT
    kp1, des1 = match_sift(image1)
    kp2, des2 = match_sift(image2)
    
    # Match the descriptors
    matches = match_bf(des1, des2)
    
    # Apply RANSAC to remove outliers and compute homography
    M, inliers = remove_outliers_ransac(kp1, kp2, matches)
    
    # Print the homography matrix for debugging
    print("Homography Matrix (M):")
    print(M)
    dx = M[0, 2]  # Horizontal translation
    dy = M[1, 2]  # Vertical translation
    print(f"Displacement Vector: dx={dx}, dy={dy}")

    print(f"Number of Matches: {len(matches)}")
    print(f"Number of Inliers: {len(inliers)}")

    # calculate the mean displacement vector for inliers
    mean_displacement = []
    for m in inliers:
        pt1 = kp1[m.queryIdx].pt
        pt2 = kp2[m.trainIdx].pt
        dx = pt2[0] - pt1[0]
        dy = pt2[1] - pt1[1]
        mean_displacement.append((dx, dy))
    mean_displacement = np.mean(mean_displacement, axis=0)
    print(f"Mean Displacement Vector: dx={mean_displacement[0]}, dy={mean_displacement[1]}")
    
    # Draw the matches between the two images
    vis = draw_matches(image1_color, kp1, image2_color, kp2, inliers)
    
    # Show the matched image visualization
    plt.imshow(cv2.cvtColor(vis, cv2.COLOR_BGR2RGB))
    plt.title("Matches")
    plt.show()

main()


Mean Displacement Vector: dx=-27.69120998800236, dy=404.1618301572591
Homography Matrix (M):
[[9.67885196e-01 9.20412534e-03 4.35780287e+00]
 [2.69676391e-02 9.40814104e-01 4.10504534e+02]
 [4.82322516e-07 3.52460233e-06 1.00000000e+00]]
Displacement Vector: dx=4.357802868561534, dy=410.5045344801554
Number of Matches: 528
Number of Inliers: 411
Mean Displacement Vector: dx=-27.69120998800236, dy=404.1618301572591


This homography matrix, M, represents the 2D transformation from the reference image (keypoints1) to the target image (keypoints2).



# using Brute force and Lowe Ratio with sift

In [101]:
import numpy as np
import cv2 as cv
from matplotlib import pyplot as plt
MIN_MATCH_COUNT = 10

img1 = cv.imread('earth_img\photo_091_53245728575_o.jpg',0)          # queryImage
img2 = cv.imread('earth_img\photo_092_53245529093_o.jpg',0) # trainImage

# Initiate SIFT detector
sift = cv.SIFT_create()

# find the keypoints and descriptors with SIFT
kp1, des1 = sift.detectAndCompute(img1,None)
kp2, des2 = sift.detectAndCompute(img2,None)

bf = cv.BFMatcher()
matches = bf.knnMatch(des1,des2, k=2)


# store all the good matches as per Lowe's ratio test.
good = []
for m,n in matches:
    if m.distance < 0.7*n.distance:
        good.append(m)

print(len(good))
print(len(matches))

img3 = cv.drawMatches(img1,kp1,img2,kp2,good,None,flags=cv.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

plt.imshow(img3),plt.show()

512
1020


(<matplotlib.image.AxesImage at 0x1ed82760280>, None)

# using FLANN and Lowe Ratio with sift

In [ ]:
import numpy as np
import cv2 as cv
from matplotlib import pyplot as plt
MIN_MATCH_COUNT = 10

img1 = cv.imread('earth_img\photo_091_53245728575_o.jpg',0)          # queryImage
img2 = cv.imread('earth_img\photo_092_53245529093_o.jpg',0) # trainImage

# Initiate SIFT detector
sift = cv.SIFT_create()

# find the keypoints and descriptors with SIFT
kp1, des1 = sift.detectAndCompute(img1,None)
kp2, des2 = sift.detectAndCompute(img2,None)

# FLANN parameters
FLANN_INDEX_KDTREE = 1
index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
search_params = dict(checks=100)   # or pass empty dictionary

flann = cv.FlannBasedMatcher(index_params,search_params)

matches = flann.knnMatch(des1,des2,k=2)

# store all the good matches as per Lowe's ratio test.
good = []
for m,n in matches:
    if m.distance < 0.7*n.distance:
        good.append(m)

print(len(good))
print(len(matches))

img3 = cv.drawMatches(img1,kp1,img2,kp2,good,None,flags=cv.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

plt.imshow(img3),plt.show()

1020
1020


(<matplotlib.image.AxesImage at 0x1ed84645ca0>, None)

Now we set a condition that at least 10 matches (defined by MIN_MATCH_COUNT) are to be there to find the object. Otherwise simply show a message saying not enough matches are present.

If enough matches are found, we extract the locations of matched keypoints in both the images. They are passed to find the perspective transformation. Once we get this 3x3 transformation matrix, we use it to transform the corners of queryImage to corresponding points in trainImage. Then we draw it. 

In [ ]:
if len(good)>MIN_MATCH_COUNT:
    src_pts = np.float32([ kp1[m.queryIdx].pt for m in good ]).reshape(-1,1,2)
    dst_pts = np.float32([ kp2[m.trainIdx].pt for m in good ]).reshape(-1,1,2)
    M, mask = cv.findHomography(src_pts, dst_pts, cv.RANSAC,5.0)
    matchesMask = mask.ravel().tolist()
    h,w = img1.shape
    print("h, w: ", h,w)
    pts = np.float32([ [0,0],[0,h-1],[w-1,h-1],[w-1,0] ]).reshape(-1,1,2)
    print("pst: ", pts)
    dst = cv.perspectiveTransform(pts,M)
    print("dts: ", dst)
    img2 = cv.polylines(img2,[np.int32(dst)],True,255,3, cv.LINE_AA)
else:
    print( "Not enough matches are found - {}/{}".format(len(good), MIN_MATCH_COUNT) )
    matchesMask = None

h, w:  3040 4056
pst:  [[[   0.    0.]]

 [[   0. 3039.]]

 [[4055. 3039.]]

 [[4055.    0.]]]
dts:  [[[  11.403623  826.8927  ]]

 [[  36.60615  3662.2334  ]]

 [[3937.8252   3763.2844  ]]

 [[3929.4214    928.07385 ]]]


Finally we draw our inliers (if successfully found the object) or matching keypoints (if failed).

In [ ]:
%matplotlib qt

draw_params = dict(matchColor = (0,255,0), # draw matches in green color
                   singlePointColor = None,
                   matchesMask = matchesMask, # draw only inliers
                   flags = 2)

print(len(good))
print(len(matches))
print(len(matchesMask))

displament_from_inliers = []
for m in good:
    pt1 = kp1[m.queryIdx].pt
    pt2 = kp2[m.trainIdx].pt
    dx = pt2[0] - pt1[0]
    dy = pt2[1] - pt1[1]
    displament_from_inliers.append((dx, dy))

displament_from_inliers = np.mean(displament_from_inliers, axis=0)
print(f"Mean Displacement Vector: dx={displament_from_inliers[0]}, dy={displament_from_inliers[1]}")

print("Homography Matrix:", M)

print("x = ",M[0, 2])
print("y = ",M[1, 2])


 
img3 = cv.drawMatches(img1,kp1,img2,kp2,good,None,**draw_params)
 
plt.imshow(img3, 'gray'),plt.show()

514
1020
514
Mean Displacement Vector: dx=-64.67359035209923, dy=809.8123959938376
Homography Matrix: [[ 9.66114712e-01  8.34490403e-03  1.14036224e+01]
 [ 2.49275749e-02  9.38174147e-01  8.26892685e+02]
 [-2.65250670e-08  1.41699668e-06  1.00000000e+00]]
x =  11.403622437005497
y =  826.8926850114964


(<matplotlib.image.AxesImage at 0x1ed82760310>, None)

2. Convert Pixel Displacement to Camera Coordinates

To relate the displacement in pixels to physical distances (in meters), you first need to convert pixel coordinates into camera coordinates. This can be done using the camera intrinsic matrix.

Given a pixel displacement vector (dx,dy)(dx,dy), you can calculate the displacement in the camera's coordinate system using the following relations:

delta X = dx/f_x

delta Y = dy/f_y

Where ΔX and ΔY are the displacement components in the camera's coordinate system (in meters).

3. Depth Information

To convert the displacement from camera coordinates to real-world coordinates (in meters), you also need depth information, which is the distance from the camera to the object in question. This step is necessary because the displacement in pixels corresponds to an angular displacement, which depends on the depth.

If you have depth ZZ (in meters), the physical displacement can be calculated as:
ΔXmeters=ΔX/Z

ΔYmeters​=ΔY/z​

Thus, the physical displacement in meters will depend on the depth ZZ, which could be obtained from stereo vision, a depth sensor, or assuming a known depth.

### Notes

- Focal Length: If you don’t have the focal length in pixels, you can estimate it from the camera's physical focal length and the sensor size. Focal length in pixels is calculated as:

f_x = f_physical * image width / sensor width

See camera calibration to get focal length

from PIL import Image
from PIL.ExifTags import TAGS

def get_focal_length(image_path):
    img = Image.open(image_path)
    exif_data = img._getexif()
    if exif_data:
        for tag, value in exif_data.items():
            if TAGS.get(tag) == "FocalLength":
                return value
    return None

print(get_focal_length("your_image.jpg"))


In [ ]:
def pixel_to_meters(dx, dy, fx, fy, depth):
    """
    Convert pixel displacement to real-world displacement in meters.
    
    Parameters:
    - dx, dy: Pixel displacement
    - fx, fy: Focal lengths in pixels (from intrinsic matrix)
    - depth: Depth of scene (meters)
    
    Returns:
    - (dx_meters, dy_meters): Displacement in meters
    """
    dx_meters = (dx * depth) / fx
    dy_meters = (dy * depth) / fy
    return dx_meters, dy_meters

FULL_FRAME_WIDTH = 4056 # pixels
FULL_FRAME_HEIGHT = 3040 # pixels
SENSOR_HIEGHT = 5.476 # mm
SENSOR_WIDTH = 7.564 # mm
FOCAL_LENGTH = 5 # mm

FOCAL_X = FOCAL_LENGTH * (FULL_FRAME_WIDTH / SENSOR_WIDTH)
FOCAL_Y = FOCAL_LENGTH * (FULL_FRAME_HEIGHT / SENSOR_HIEGHT)
PRINCIPLE_POINT_X = FULL_FRAME_WIDTH / 2
PRINCIPLE_POINT_Y = FULL_FRAME_HEIGHT / 2

CAMERA_K = np.array([[FOCAL_X, 0, PRINCIPLE_POINT_X], 
              [0, FOCAL_Y, PRINCIPLE_POINT_Y], 
              [0, 0, 1]])

DEPTH_Z = 420000 #(4.2e+05) meters

dx_meters, dy_meters = pixel_to_meters(displament_from_inliers[0], displament_from_inliers[1], FOCAL_X, FOCAL_Y, DEPTH_Z)

print(f"Displacement in meters: dx={dx_meters}, dy={dy_meters}")

dx_meters_homography, dy_meters_homography = pixel_to_meters(M[0, 2], M[1, 2], FOCAL_X, FOCAL_Y, DEPTH_Z)

print(f"Displacement in meters (Homography): dx={dx_meters_homography}, dy={dy_meters_homography}")

# extract time difference between images using exif data


# convert to kilometers and combine for magnitude
magnitude = np.sqrt(dx_meters**2 + dy_meters**2) / 1000
print(f"Magnitude of Displacement: {magnitude:.2f} km")

magnitude_homography = np.sqrt(dx_meters_homography**2 + dy_meters_homography**2) / 1000
print(f"Magnitude of Displacement (Homography): {magnitude_homography:.2f} km")

Displacement in meters: dx=-10131.175331251332, dy=122533.1398548781
Displacement in meters (Homography): dx=1786.3875763152873, dy=125117.56737576585
Magnitude of Displacement: 122.95 km
Magnitude of Displacement (Homography): 125.13 km


In [ ]:
%pip install exif

from exif import Image
from datetime import datetime

def get_time(image):
    with open(image, 'rb') as image_file:
        img = Image(image_file)
        time_str = img.get("datetime_original")
        time = datetime.strptime(time_str, '%Y:%m:%d %H:%M:%S')
    return time

time1 = get_time(IMAGE_COMPARISON)
time2 = get_time(IMAGE_REFERENCE)
time_diff = time2 - time1
print(f"Time Difference: {time_diff}")

Note: you may need to restart the kernel to use updated packages.
Time Difference: 0:00:16


In [ ]:
print(f"velocity: {magnitude / time_diff.total_seconds()} km/s")

print(f"velocity (Homography): {magnitude_homography / time_diff.total_seconds()} km/s")

velocity: 7.684453429929074 km/s
velocity (Homography): 7.820644965860523 km/s
